[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/broad-1/quickstarters/random-submission/random-submission.ipynb)

![Cover](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/broad-1/assets/cover.png)

Change in load_data()

In [ ]:
%pip install --upgrade crunch-cli

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of distributed to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort

Get a new token: https://hub.crunchdao.com/competitions/broad-1/submit/via/notebook

In [ ]:
!crunch setup --help

Usage: crunch setup [OPTIONS] COMPETITION_NAME PROJECT_NAME [DIRECTORY]

  Setup a workspace directory with the latest submission of you code.

Options:
  --token TEXT                   Clone token to use.  [required]
  --submission INTEGER           Submission number to clone. (latest if not
                                 specified)
  --no-data                      Do not download the data. (faster)
  --no-model                     Do not download the model of the cloned
                                 submission.
  -f, --force                    Deleting the old directory (if any).
  --model-directory TEXT         Directory where your model is stored.
                                 [default: resources]
  --no-quickstarter              Disable quickstarter selection.
  --quickstarter-name TEXT       Pre-select a quickstarter.
  --show-notebook-quickstarters  Show quickstarters notebook in selection.
  --notebook                     Setup everything for a notebook environment.
  -

In [ ]:
# !crunch setup --notebook --size default broad-1 test --token tPM1wCmTDAChUfvdjixZEGdh9BvAH39jHXq1aG8Xg8UVMyvs64TjPQLjV90hkVDq

main.py: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/15254/main.py (57629 bytes)
notebook.ipynb: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/15254/notebook.ipynb (77431 bytes)
requirements.txt: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/15254/requirements.original.txt (302 bytes)
resources/checkpoint_epoch_219.pth.tar: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/models/16455/checkpoint_epoch_219.pth.tar (255794751 bytes)
resources/pytorch_model.bin: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/models/16455/pytorch_model.bin (1213527781 bytes)
data/DC1.zarr.zip: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/81/DC1.zarr.zip (720810785 bytes)
data/DC1.zarr.zip: uncompress into data/DC1.zarr.zip.7l51mdy1
data/UC1_NI.za

## Preprocess

In [ ]:
import crunch
crunch = crunch.load_notebook()

### Save pre_load of cells for each image ( position, id and GE)

In [ ]:
# !pip install spatialdata

In [ ]:
import os
import spatialdata as sd
import pickle
import numpy as np
from skimage.measure import regionprops
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import itertools

from collections import defaultdict

In [ ]:
# sdata_dir=f'F:/DATA/crunch_large/zip_server

# pre_load_path= './tmp/pre_load'

# os.makedirs(pre_load_path,exist_ok=True)

In [ ]:
# dir = f'D:/data/crunch_large/data/'
def create_pre_load(pre_load_path,sdata_dir= '/data',name_list=['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']):
    NAMES = name_list
    # names = sample_names[2:]
    for name in NAMES:
        pkl_file_path= f'{pre_load_path}/{name}_cells.pkl'
        if os.path.exists(pkl_file_path):
            continue
        with open(pkl_file_path,'wb') as f:
            print(f"{sdata_dir}/{name}.zarr")
            sdata = sd.read_zarr(f"{sdata_dir}/{name}.zarr")
            cell_id_group=sdata['cell_id-group']

        # anucleus= sdata['anucleus']

            cell_id_train =  cell_id_group.obs[ cell_id_group.obs['group'] == 'train']['cell_id'].to_numpy()
            # cell_id_train = list(set(cell_id_train).intersection(set( anucleus.obs['cell_id'].unique())))

            cell_id_test= cell_id_group.obs[ cell_id_group.obs['group'] == 'test']['cell_id']
            cell_id_validation= cell_id_group.obs[ cell_id_group.obs['group'] == 'validation']['cell_id']
            max_len= max(max(cell_id_train),max(cell_id_test),max(cell_id_validation))
            bool_list= [0]*(max_len+1)

            for id in cell_id_train:
                bool_list[id]=1
            for id in cell_id_test:
                bool_list[id]=2
            for id in cell_id_validation:
                bool_list[id]=3
            ground_truth=np.array([])
            if name != 'DC1':
                anucleus= sdata['anucleus']
                cell_id_train = list(set(cell_id_train).intersection(set( anucleus.obs['cell_id'].unique())))
                ground_truth =  anucleus.layers['counts'][ anucleus.obs['cell_id'].isin(cell_id_train),:]
            r=10
            # im= sdata['HE_registered'].to_numpy()
            patches_list = []  # Initialize an empty list to store patches
            index=0

            cell_list=[]
            for props in tqdm( regionprops(sdata['HE_nuc_original'][0, :, :].to_numpy()) ):
                cell_item={}
                cell_item['cell_id']= props.label
                centroid = props.centroid
                cell_item['center']=[int(centroid[0]), int(centroid[1])]
                if cell_item['cell_id'] >= len(bool_list):
                    cell_item['label']='None'
                    cell_item['anucleus']=np.array([])
                    continue
                elif bool_list[props.label] ==1:
                    if len(ground_truth)!=0:
                        cell_item['anucleus']=ground_truth[index]
                        index+=1
                    else:
                        cell_item['anucleus']=np.array([])
                    cell_item['label']='train'
                elif bool_list[props.label] ==2:
                    cell_item['anucleus']=np.array([])
                    cell_item['label']='test'
                elif bool_list[props.label] ==3:
                    cell_item['anucleus']=np.array([])
                    cell_item['label']='validation'
                else:
                    cell_item['anucleus']=np.array([])
                    cell_item['label']='None'
                cell_list.append(cell_item)
            sdata=None
            pickle.dump(cell_list,f)

### Create cluster cells for each H&E

In [ ]:
# cluster_path= './tmp/cluster'


In [ ]:
def create_cluster(group,n_centroid,NAMES,pre_load_path,cluster_path):
    for name in NAMES:
        with open(f'{pre_load_path}/{name}_cells.pkl','rb') as f:
            cell_list= pickle.load(f)

        if group =='evel':
            filtered_cells = [c for c in cell_list if c['label'] in ('validation', 'test')]
        else:
            filtered_cells = [c for c in cell_list if c['label'] == group]

        cell_locations = pd.DataFrame([{'x': c['center'][1], 'y': c['center'][0],'cell_id': c['cell_id'],'counts':c['anucleus']} for c in filtered_cells])

        # Define the number of clusters
        n_clusters = n_centroid  # Adjust this based on your data and requirements

        # Perform K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(cell_locations[['x', 'y']])

        # Add the cluster labels to the DataFrame
        cell_locations['cluster'] = kmeans.labels_

        # Visualize the clusters
        plt.figure(figsize=(10, 10))
        plt.scatter(
            cell_locations['x'],
            cell_locations['y'],
            c=cell_locations['cluster'],
            s=0.5,
            cmap='viridis',  # Continuous colormap
            alpha=0.5
        )
        plt.colorbar(label='Cluster ID')  # Add colorbar to interpret color mapping
        plt.title('Clusters Visualized with Continuous Colormap')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.show()
        dir= f'{cluster_path}/{group}/cluster_data'
        os.makedirs(f'{dir}',exist_ok= True )
        with open(f'{dir}/{name}_cells.pkl','wb') as f:
            pickle.dump(cell_locations,f)
        with open(f'{dir}/{name}_kmeans.pkl','wb') as f:
            pickle.dump(kmeans,f)
        data = cell_locations[['x', 'y']].to_numpy()
        clusters = cell_locations['cluster'].to_numpy()
        centroids = kmeans.cluster_centers_

        # Initialize a dictionary to store the radius of each cluster
        cluster_radii = {}

        # Calculate radius for each cluster
        for cluster_id in range(len(centroids)):
            # Get points belonging to the current cluster
            cluster_points = data[clusters == cluster_id]

            # Calculate distances to the cluster centroid
            distances = np.linalg.norm(cluster_points - centroids[cluster_id], axis=1)

            # Radius is the maximum distance
            cluster_radii[cluster_id] = np.max(distances)

        # Print radii for all clusters
        list_r=[]
        for cluster_id, radius in cluster_radii.items():
            print(f"Cluster {cluster_id}: Radius = {radius}")
            list_r.append(radius)
        print(f'mean radius: {np.mean(list_r)}')

#### TRAIN

In [ ]:
# create_cluster('train',n_centroid=6500,)
print('a')
def split_cluster(cluster_path,NAMES):
      group='train'
      for name in NAMES:
            dir= f'{cluster_path}/{group}/cluster_data'
            with open(f'{dir}/{name}_cells.pkl','rb') as f:
                  cell_locations=pickle.load(f)
            with open(f'{dir}/{name}_kmeans.pkl','rb') as f:
                  kmeans=pickle.load(f)
            data = cell_locations[['x', 'y']].to_numpy()
            clusters = cell_locations['cluster'].to_numpy()
            centroids = kmeans.cluster_centers_

            cell_locations['train']=[1]*kmeans.labels_.shape[0]
            # Initialize a dictionary to store the radius of each cluster
            valid_cluster_id=[]
            valid_centroid=[]
            # Calculate radius for each cluster
            for cluster_id in range(len(centroids)):
                  x_center, y_center = centroids[cluster_id]

                  half_side = int(32 / 2)
                  x_min, x_max = x_center - half_side, x_center + half_side
                  y_min, y_max = y_center - half_side, y_center + half_side

                  index_cells_list_in_square =(
                  (cell_locations['x'] >= x_min) & (cell_locations['x'] <= x_max) &
                  (cell_locations['y'] >= y_min) & (cell_locations['y'] <= y_max)
                  )
                  # Get points belonging to the current cluster
                  cluster_points = data[clusters == cluster_id]

                  # Calculate distances to the cluster centroid
                  if len(cluster_points)<=5 or len(index_cells_list_in_square)==0:
                        cell_locations.loc[cell_locations['cluster'] == cluster_id, 'train']=-1
                  # valid_cluster_id.append(cluster_id)
                  # print(len(cluster_points))
            unique_cluster_ids = cell_locations.loc[cell_locations['train'] == 1, 'cluster'].unique()

            split = np.random.choice(unique_cluster_ids, int(0.1*len(unique_cluster_ids)), replace=False)
            for cluster_id in split:
                  cell_locations.loc[cell_locations['cluster'] == cluster_id, 'train']=0
            split_dir=  f'{cluster_path}/{group}/cluster_data_split'
            os.makedirs(split_dir,exist_ok= True )
            with open(f'{split_dir}/{name}_cells.pkl', 'wb') as f:
                  pickle.dump(cell_locations, f)
            with open(f'{split_dir}/{name}_kmeans.pkl','wb') as f:
                  pickle.dump(kmeans,f)


a


#### Validation & Test

In [ ]:
# create_cluster('validation',n_centroid=600)
# create_cluster('test',n_centroid=600)

## Crop patches

In [ ]:
from PIL import Image

In [ ]:
# crop_path='./tmp/patches'
# r=80/2
def crop_patches(sdata_dir, pre_load_path,crop_path,r,name_list=['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I'],evel=False):
    NAMES = name_list
    if evel == True:
        group = 'evel'
    else:
        group= 'train'
    for name in NAMES:

        r=int(r)
        save_dir= f'{crop_path}/{r*2}'
        os.makedirs(f'{save_dir}/{group}',exist_ok=True)
        if os.path.exists(f'{save_dir}/{group}/{name}.npy'):
            continue
        with open(f'{pre_load_path}/{name}_cells.pkl','rb') as f:
                    cell_list= pickle.load(f)

        sdata = sd.read_zarr(f"{sdata_dir}/{name}.zarr")
        im= sdata['HE_original'].to_numpy()
        patches_list = []  # Initialize an empty list to store patches


        for props in cell_list:
            if (evel== False and props['label'] == 'train') \
                or (evel == True and props['label'] in ('validation', 'test')):

                cell_id= props['cell_id']
                centroid = props['center']

                x_center, y_center = centroid[1], centroid[0]
                x_center= int(x_center)
                y_center= int(y_center)
                        # Calculate the crop boundaries


                minr, maxr = y_center - r, y_center + r
                minc, maxc = x_center - r, x_center + r

                # Ensure boundaries are within the image dimensions

                if (minr <0) or (minc <0) or (maxr >im.shape[1]) or (maxc >im.shape[2]):
                    pad_top = max(0, -minr)
                    minr = max(0, minr)

                    pad_bottom = max(0, maxr - im.shape[1])
                    maxr = min(maxr, im.shape[1])

                    pad_left = max(0, -minc)
                    minc = max(0, minc)

                    pad_right = max(0, maxc - im.shape[2])
                    maxc = min(maxc, im.shape[2])

                # Crop and pad the image if needed

                    patch = np.pad(im[:, minr:maxr, minc:maxc],
                                ((0, 0), (pad_top, pad_bottom), (pad_left, pad_right)),
                                mode='constant', constant_values=0)
                else:
                    # print(minr,maxr,minc,maxc)
                    patch = im[:, minr:maxr, minc:maxc]
                    print(patch.shape)
                # patch = im[:, minr:maxr, minc:maxc]

                patch = Image.fromarray(np.transpose(patch,(2,1,0)))
                if patch.size !=(32,32):
                    patch=patch.resize((32,32))
                # except:
                #     print( minr,maxr,minc,maxc, im.shape)

                # patch = transforms.ToTensor()(patch)
                patches_list.append(patch)
        print(len(patches_list))
        patches_tensor= np.stack(patches_list)
        print(f'Patches: {patches_tensor.shape}')
        save_dir= f'{crop_path}/{r*2}'
        os.makedirs(f'{save_dir}/{group}',exist_ok=True)
        np.save(f'{save_dir}/{group}/{name}.npy',patches_tensor)
        im=None
        patches_tensor=None
        sdata=None
        del im,patches_tensor
        print(f"Saved {len(patches_list)} patches to the list.")


In [ ]:
# np.save(f'{save_dir}/{group}/{name}.npy',patches_tensor)

## Process Patches

You will need to download public pretrain model from https://huggingface.co/MahmoodLab/UNI (file: pytorch_model.bin)

In [ ]:
# pretrain_model_path='./resource/pretrain/pytorch_model.bin'
# preprocessed_path= './preprocessed'

In [ ]:
import torchvision.transforms as transforms
import torch #1.13.1
import random
from torch.utils.data import Dataset
import torch.nn as nn
import timm
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.loader import DataLoader


In [ ]:

class DataGenerator(Dataset):
    def __init__(self, numpy_folder, augmentation=True, random_seed=1234, k=5, split= False,
                 list= ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']):
        self.augmentation = augmentation
        # loc_list = sorted([os.path.basename(name) for name in glob.glob(os.path.join(image_folder, '*'))])
        # random.seed(random_seed)
        # random.shuffle(loc_list)

        # partitions = self.partition(loc_list, k)
        # selected_loc_list = [i for m in m_list for i in partitions[m]]

        # self.image_list = [
        #     img for loc in selected_loc_list for img in glob.glob(os.path.join(image_folder, loc, '*.png'))
        # ]
        NAMES = list
        # NAMES= NAMES[:1]

        tensors_list=[
                np.load(f'{numpy_folder}/{name}.npy') for name in NAMES
    ]
        # except:
        #     tensors_list=[
        #          torch.load(f'{numpy_folder}/{name}.pt') for name in NAMES
        # ]
        tensors_list= np.vstack(tensors_list)
        # for num in tensors_list:
        #     num= transforms.ToTensor()(num)
        # tensors_list=torch.from_numpy(tensors_list)
        self.tensors_list = tensors_list

    def __len__(self):
        return len(self.tensors_list)

    def __getitem__(self, index):
        tensor =transforms.ToTensor()(self.tensors_list[index])

        if self.augmentation:
            # Flip upside-down (vertically)
            if random.random() < 0.5:
                tensor = torch.flip(tensor, dims=[1])  # Flip along the height dimension

            # Flip left-to-right (horizontally)
            if random.random() < 0.5:
                tensor = torch.flip(tensor, dims=[2])

        # img = torch.from_numpy(img).permute(2, 0, 1)  # HWC to CHW
        return tensor

    @staticmethod
    def partition(lst, n):
        division = len(lst) / float(n)
        return [lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n)]
class ImageEncoder(nn.Module):
    def __init__(self,resources_path):
        super().__init__()
        model = timm.create_model(
    "vit_large_patch16_224", img_size=224, patch_size=16, init_values=1e-5, num_classes=0, dynamic_img_size=True
)
        pretrain_model_path=f'{resources_path}/pytorch_model.bin'
        model.load_state_dict(torch.load((f'{pretrain_model_path}')), strict=True)
        # model.load_state_dict(torch.load(("D:/Downloads/pytorch_model.bin"), map_location="cuda:0"), strict=True)
        self.model=model
        # self.model = nn.Sequential(*list(self.model.children())[:-1])

        for p in self.model.parameters():
            p.requires_grad = False

    def forward(self, x):
        # x=x.unsqueeze(0) # for testing single image
        # print(x.shape)
        x = self.model(x)

        return x

In [ ]:
# tensor_folder = f'{crop_path}/{int(r*2)}'
# save_dir=f'{preprocessed_path}/{int(r*2)}'


In [ ]:
# NAMES = ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']
def preprocess_patches(NAMES,tensor_folder,save_dir,resources_path='/resources',evel= False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    encoder= ImageEncoder(resources_path=resources_path)
    encoder.eval()
    encoder=encoder.to(device)
    emb_stack=[]
    # from collections import defaultdict



    # local_index = 0
    # label_map = defaultdict(list)

    for name in NAMES:


        if evel==False:
            label= 'train'
        else:
             label= 'evel'

        if os.path.exists(f'{save_dir}/{label}/{name}.npy'):
            continue

        data_dir=tensor_folder+f'/{label}'
        dataset = DataGenerator(data_dir,list=[name])
        # print(dataset[[0,1,2]],indices[train_mask].shape)

        comb_set={}


        emb_stack=[]
        # if label== 'test' and name== NAMES[5]:
        #     continue
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=200, shuffle=False)

        with torch.no_grad():
            i=0
            for batch in dataloader:
                batch = batch.to(device)
                emb= encoder(batch)
                emb_stack.append(emb.cpu())
                i+=1
                if i%10 ==0:
                    print(i)

        emb_stack = torch.cat(emb_stack, dim=0)
        print(label,emb_stack.shape)
        emb_stack=emb_stack.numpy()
        os.makedirs(f'{save_dir}/{label}',exist_ok=True)
        np.save(f'{save_dir}/{label}/{name}.npy',emb_stack)

In [ ]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from types import SimpleNamespace
from torch_geometric.data import Data
from scipy.spatial import distance
from scipy.spatial.distance import cdist

def get_edge_index(dataframe, k=6):
    # """
    # Calculate edge indices for a k-neighbor graph based on distances.
    
    # Args:
    #     dataframe (pd.DataFrame): DataFrame with 'x' and 'y' columns.
    #     k (int): Number of neighbors (default is 6).
    
    # Returns:
    #     edge_index (list of tuples): List of edges as (source, target).
    # """
    # Extract coordinates
        coordinates = dataframe[['x', 'y']].to_numpy()  # Shape: (n, 2)
        
        # Calculate pairwise distances
        dist_matrix = distance.cdist(coordinates, coordinates, metric='euclidean')  # Shape: (n, n)
        
        # For each point, find indices of k nearest neighbors (excluding itself)
        n = len(coordinates)
        edge_index = []
        for i in range(n):
            # Get indices of sorted distances (excluding itself)
            neighbors = np.argsort(dist_matrix[i])[1:k+1]
            
            # Create edges from i to each neighbor
            for neighbor in neighbors:
                edge_index.append((i, neighbor))  # (source, target)
                edge_index.append((neighbor, i))
            edge_index.append((-1,i))
        # edge_index= np.array(edge_index)
        
        # edge_index+=1
        return edge_index


class NeuronData_3(Dataset):
    def __init__(self,cluster_path= 'E:/DATA/crunch/tmp', emb_folder=f'D:/DATA/Gene_expression/Crunch/preprocessed'
                 , augmentation=True,encoder_mode=False, random_seed=1234, train=True, split= False,
                 name_list= ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I'],evel=False):
        self.augmentation = augmentation
        emb_dir=emb_folder
        # emb_dir=  
        self.encoder_mode= encoder_mode
        NAMES = name_list
        group='evel'
        dataset_type= None
        if train== False and split == True:
                dataset_type= 0 
        elif train== True and split == True:
                dataset_type= 1
        elif train== True and split == False:
                dataset_type= -1
        if split ==True:
            group='train'
            try:
                NAMES.remove('DC1')
            except:
                print('DC1 is not in list')
        if evel ==True:
            group='evel'
            dataset_type=-1
        # NAMES= NAMES[:1]
        self.group= group
        print(f'Type: {dataset_type}, group: {group}')
        centroids_dict={}
        valid_clusters_dict={}
        valid_cell_list_cluster_dict={}
        emb_cells_dict={}
        emb_centroids_dict={}
        lenngths=[]
        cluster_edge_index_dict={}
        cluster_index_cells_list_in_square_dict={}
        
        cluster_centroid_exps_dict={}
        cluster_cell_exps_dict={}
        cluster_cell_mask_dict={}
        sorted_indices_dict={}
        cell_id_list_dict={}
        for name in NAMES:
            print(name)
            cluster_edge_index=[]
            cluster_index_cells_list_in_square=[]
            cluster_centroid_exps=[]
            cluster_cell_exps=[]
            cluster_cell_mask=[]
            cell_id_list=[]
      
            if split== True:
                cluster_dir=f'{cluster_path}/cluster/{group}/cluster_data_split'
            elif train == False:
                cluster_dir=f'{cluster_path}/cluster/{group}/cluster_data'
                
            with open(f'{cluster_dir}/{name}_cells.pkl','rb') as f:
                cell_list_cluster = pickle.load(f)
          
            
            emb_cells= torch.from_numpy(np.load(f'{emb_dir}/24/{group}/{name}.npy'))
            emb_centroids= torch.from_numpy(np.load(f'{emb_dir}/80/{group}/{name}.npy')) # len of valid_clusters(['group'] != -1) = len of emb_centroids

            # centroids = kmeans.cluster_centers_ # n x [x,y]
            centroids = cell_list_cluster.groupby('cluster')[['x', 'y']].mean().sort_index().reset_index().to_numpy()
            # Filter out invalid clusters (those with 'train' = -1)
            distances = cdist(centroids, centroids)

            # Initialize variables
            
            if group =='train':
                valid_cell_list_cluster= cell_list_cluster[cell_list_cluster[group] != -1]
                # len of emb_cells == len of cell_list_cluster
                emb_cells= emb_cells[cell_list_cluster[group] != -1] # len of emb_cells == len of valid_cell_list_cluster
            else:
                valid_cell_list_cluster= cell_list_cluster
                
            valid_clusters = valid_cell_list_cluster['cluster'].unique()    
            # filtered_cells_index = [i for i in range(len(cell_list_org)) if cell_list_org[i]['label'] == group]
            
   
            if dataset_type != None:
                if dataset_type ==1 :
                    temp_cluster            =cell_list_cluster[cell_list_cluster[group] == 1]['cluster'].unique()
                    cluster_index           =[list(valid_clusters).index(cluster_id) for cluster_id in temp_cluster ]
                    emb_centroids           =emb_centroids[cluster_index]
                    emb_cells               =emb_cells[valid_cell_list_cluster[group].to_numpy() == 1]
                    valid_cell_list_cluster =valid_cell_list_cluster[valid_cell_list_cluster[group] == 1]
                    valid_clusters          =temp_cluster
                elif dataset_type ==0:
                    temp_cluster            =cell_list_cluster[cell_list_cluster[group] == 0]['cluster'].unique()
                    cluster_index           =[list(valid_clusters).index(cluster_id) for cluster_id in temp_cluster ]
                    emb_centroids           =emb_centroids[cluster_index]
                    emb_cells               =emb_cells[valid_cell_list_cluster[group].to_numpy() == 0]
                    valid_cell_list_cluster =valid_cell_list_cluster[valid_cell_list_cluster[group] == 0]
                    valid_clusters          =temp_cluster
          
            centroids= centroids[valid_clusters]   
            ''''''
            n_centroids = len(centroids)
            visited = [False] * n_centroids
            sorted_indices = []
            current = 0  # Start from the first centroid

            for _ in range(n_centroids):
                # Mark current centroid as visited
                visited[current] = True
                sorted_indices.append(current)

                # Find the next closest unvisited centroid
                unvisited_distances = [(i, distances[current, i]) for i in range(n_centroids) if not visited[i]]
                if unvisited_distances:
                    current = min(unvisited_distances, key=lambda x: x[1])[0]

            # Use the sorted indices to get sorted centroids
            sorted_indices_dict[name]=sorted_indices
            ''''''
            lenngths.append(len(centroids))
            centroids_dict[name]            =centroids
            valid_clusters_dict[name]       =valid_clusters
            emb_cells_dict[name]            =emb_cells
            emb_centroids_dict[name]        =emb_centroids
            #####################################################
            
            for i in range(len(valid_clusters)):
                cell_ids= valid_cell_list_cluster['cell_id'].to_numpy()
                cell_id_list.append(cell_ids)
                cell_mask=(valid_cell_list_cluster['cluster'].to_numpy() == valid_clusters[i])
                cluster_cell_mask.append(cell_mask)
                
                cells_list_in_cluster = valid_cell_list_cluster[valid_cell_list_cluster['cluster'] == valid_clusters[i]]
                edge_index=get_edge_index(cells_list_in_cluster,k=6)
                cluster_edge_index.append(edge_index)
                x_center, y_center = centroids[i,1],centroids[i,2]
                if group == 'train':
                    half_side = int(256 / 2)
                    x_min, x_max = x_center - half_side, x_center + half_side
                    y_min, y_max = y_center - half_side, y_center + half_side
                    
                    index_cells_list_in_square =(
                    (cells_list_in_cluster['x'] >= x_min) & (cells_list_in_cluster['x'] <= x_max) &
                    (cells_list_in_cluster['y'] >= y_min) & (cells_list_in_cluster['y'] <= y_max)
                    )
                    
                    
                    ############
                    cells_list_in_square=cells_list_in_cluster[index_cells_list_in_square]
                    if len(cells_list_in_square)==0:
                        cells_list_in_square=cells_list_in_cluster
                    centroid_exps=  np.array([np.sum(cells_list_in_square['counts'].to_numpy()/len(cells_list_in_square), axis=0)])
                    normalized_counts = centroid_exps / centroid_exps.sum(axis=1, keepdims=True) * 100
                    centroid_exps = np.log1p(normalized_counts)
                    cells_list_in_square=None
                    del cells_list_in_square
                    
                    cell_exps= np.stack(cells_list_in_cluster['counts'].to_numpy())
                    normalized_counts = cell_exps / cell_exps.sum(axis=1, keepdims=True) * 100
                    cell_exps = np.log1p(normalized_counts)
                    
                    cells_list_in_cluster=None
                    del cells_list_in_cluster
                else:
                    centroid_exps=np.empty(460)
                    cell_exps=np.empty(460)
                    
                
                
                
                cluster_centroid_exps.append(centroid_exps) 
                centroid_exps=None
                cluster_cell_exps.append(cell_exps)
                cell_exps=None
                del centroid_exps,cell_exps
            
            cell_id_list_dict[name]=cell_id_list
            cluster_cell_mask_dict[name]=cluster_cell_mask    
            cluster_centroid_exps_dict[name]= cluster_centroid_exps
            cluster_cell_exps_dict[name]= cluster_cell_exps
            
            cluster_centroid_exps=None
            cluster_cell_exps=None
            valid_cell_list_cluster=None
            cluster_cell_mask=None
            del valid_cell_list_cluster,cluster_cell_mask
                
                ##############
                
            cluster_index_cells_list_in_square_dict[name]=cluster_index_cells_list_in_square
            cluster_edge_index_dict[name]= cluster_edge_index
            cluster_edge_index=None
            cluster_index_cells_list_in_square=None
               
        self.cell_id_list_dict      = cell_id_list_dict 
        self.cluster_cell_mask_dict= cluster_cell_mask_dict
        self.cluster_cell_exps_dict = cluster_cell_exps_dict
        self.cluster_centroid_exps_dict= cluster_centroid_exps_dict
        self.cluster_edge_index_dict= cluster_edge_index_dict
        self.centroids_dict             =centroids_dict
        self.valid_clusters_dict        =valid_clusters_dict
        self.emb_cells_dict             =emb_cells_dict
        self.emb_centroids_dict         =emb_centroids_dict
        self.lengths                    =lenngths
        self.sorted_indices_dict        =sorted_indices_dict
        self.cumlen =np.cumsum(self.lengths)
        self.id2name = dict(enumerate(NAMES))
        
        cluster_cell_mask_dict=None
        cluster_cell_exps_dict=None
        cluster_centroid_exps_dict=None
        valid_cell_list_cluster=None
        cell_list_cluster   =None
        emb_cells           =None
        emb_centroids       =None
        self.global_to_local = self._create_global_index_map()
    def __len__(self):
        return self.cumlen[-1]
    
    def _create_global_index_map(self):
        """Create a mapping from global index to (dataset index, local index)."""
        global_to_local = []
        for i, name in enumerate(self.id2name):
            local_indices = list(zip([i] * self.lengths[i], range(self.lengths[i])))
            global_to_local.extend(local_indices)
        return global_to_local
    
    def __getitem__(self, index):
        
     
        i, idx = self.global_to_local[index]
        idx= self.sorted_indices_dict[self.id2name[i]][idx]
        centroid= self.centroids_dict[self.id2name[i]][idx]
        cluster_id= self.valid_clusters_dict[self.id2name[i]][idx]
        cell_ids= self.cell_id_list_dict[self.id2name[i]][idx]
        
        emb_centroid=self.emb_centroids_dict[self.id2name[i]][idx]
        cluster_edge_index= self.cluster_edge_index_dict[self.id2name[i]][idx]
        
        if self.group =='train':
            cell_exps=self.cluster_cell_exps_dict[self.id2name[i]][idx]
        else:
            cell_exps=np.empty(460)
        centroid_exps= self.cluster_centroid_exps_dict[self.id2name[i]][idx]
        cluster_cell_mask= self.cluster_cell_mask_dict[self.id2name[i]][idx]
        emb_cells_in_cluster= self.emb_cells_dict[self.id2name[i]][cluster_cell_mask]
      
    
        
        cell_edge_index= cluster_edge_index
      
        # return item
        if self.encoder_mode==True:
            emb_cells_in_cluster=torch.empty(0)
            cell_exps=np.array([])
            cell_edge_index=[]
        return  Data(
            x=emb_cells_in_cluster,  # Node features (including centroid and cells)
            edge_index=cell_edge_index,  # Edge indices (intra-cluster and centroid-to-cell)
            emb_centroid=emb_centroid,  # Embedding for centroid
            cluster_centroid=centroid, # Centroid coordinates
            cell_exps=cell_exps,
            centroid_exps=centroid_exps,
            cell_num= len(emb_cells_in_cluster),
            cell_ids= cell_ids
        )
        # return emb_centroid, GE_centroid, centroid, emb_cells_in_cluster, cell_exps, cell_edge_index

       

    # @staticmethod
    # def partition(lst, n):
    #     division = len(lst) / float(n)
    #     return [lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n)]
 
def build_batch_graph(batch,device,centroid_layer):
    """Build a graph for a batch of 400 clusters."""
    all_x = []  # Store node features
    all_edge_index = []  # Store edges
    cluster_centroids = []  # Store centroids for all clusters
    centroid_exps=[]
    cell_exps=[]
    all_centroids_emb=[]
    node_offset = 0  # To adjust indices for each cluster
    # print(batch.x.shape)
    centroid_exps = np.vstack(batch.centroid_exps)  # Get centroid expression data
    # print(centroid_exps.shape)
    # print(batch.x.shape, batch.emb_centroid)
    if batch.x.shape[0]!=0:
        cell_exps = np.vstack(batch.cell_exps)  # Get cell expression data
        # print(centroid_exps.shape, cell_exps.shape)
        exps= np.vstack((centroid_exps, cell_exps))
        for i in range(len(batch.edge_index)):
            # print(torch.tensor(batch.edge_index[i]).shape)
            all_edge_index.append(torch.tensor(batch.edge_index[i]) + node_offset + len(batch.edge_index))
            # print(torch.tensor(edge_index).shape,len(edge_index))
            node_offset += batch.cell_num[i]
        edge_index = torch.cat(all_edge_index, dim=0)
        # emb_centroids=batch.emb_centroid
        batch.emb_centroid= batch.emb_centroid.view(-1,1024)
        all_x = torch.cat([batch.emb_centroid, batch.x], dim=0)
    else:
        exps=centroid_exps
        all_edge_index = []
        # print(len(edge_index))
        # print(edge_index.size(0),batch.edge_index.size(0))
        edge_index=torch.empty(0, dtype=torch.long)
        all_x = batch.emb_centroid
    # print(all_x.shape,centroid_exps.shape)
    # print(edge_index.shape)
    # all_x = batch.x  # All node features (batch size x feature_dim)
    

    # Assuming centroid embeddings are stored as a separate tensor
    # emb_centroids = batch.emb_centroid  # Stack centroid embeddings from the batch
    cluster_centroids = torch.tensor(np.array(batch.cluster_centroid))  # Get centroid positions
    
   
    dist_matrix = torch.cdist(cluster_centroids, cluster_centroids)
    
  
    k=min(6,len(cluster_centroids))
    neighbors = torch.topk(dist_matrix, k=k, largest=False).indices  # Top 6 neighbors

    # Add inter-cluster edges
    edge_index_centroid = []
    for i in range(len(cluster_centroids)):
        add= torch.stack([torch.full((k,), i), neighbors[i]])
        edge_index_centroid.append(add)
    edge_index_centroid = torch.cat(edge_index_centroid, dim=1)
    edge_index_centroid= edge_index_centroid.T
    edge_index = torch.cat([edge_index, edge_index_centroid], dim=0)
    if centroid_layer ==False:
        edge_index_centroid =None
   
    return Data(x=all_x.to(device), edge_index=edge_index.to(device), edge_index_centroid=edge_index_centroid),exps,centroid_exps


class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text


class LR_Scheduler(_LRScheduler):
    def __init__(self, optimizer:Optimizer,
                 num_epochs:int, iter_per_epoch:int, warmup_epochs:int,
                 warmup_lr:float, base_lr:float, final_lr:float,
                 constant_predictor_lr=False):

        assert num_epochs > 0 and iter_per_epoch > 0 and warmup_epochs >= 0
        assert warmup_epochs <= num_epochs
        assert warmup_lr >= 0 and base_lr >= 0 and final_lr >= 0

        self.base_lr = base_lr
        self.constant_predictor_lr = constant_predictor_lr

        warmup_iter = iter_per_epoch * warmup_epochs
        warmup_lr_schedule = np.linspace(warmup_lr, base_lr, warmup_iter)
        decay_iter = iter_per_epoch * (num_epochs - warmup_epochs)
        cosine_lr_schedule = final_lr + 0.5*(base_lr - final_lr) * (1 + np.cos(np.pi*np.arange(decay_iter) / decay_iter))

        self.lr_schedule = np.concatenate((warmup_lr_schedule, cosine_lr_schedule))
        self.optimizer = optimizer
        self.iter = -1
        self.current_lr = 0

        super().__init__(optimizer)

    def step(self):
        for param_group in self.optimizer.param_groups:

            if self.constant_predictor_lr and param_group['name'] == 'predictor':
                param_group['lr'] = self.base_lr
            else:
                lr = param_group['lr'] = self.lr_schedule[self.iter]

        self.iter += 1
        self.current_lr = lr
        return lr

    def get_lr(self):
        return self.current_lr
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]




## TRAIN

In [ ]:
def save_checkpoint(epoch, model, optimizer,scheduler, args, filename="checkpoint.pth.tar"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler': scheduler.state_dict(),
        'args': args
    }
    dir=f"{args.save_dir}"
    if args.encoder_mode== True:
        dir=f"{args.save_dir}/encoder"
    os.makedirs(dir, exist_ok=True)
    torch.save(checkpoint, f"{dir}/{filename}")
    print(f"Checkpoint saved at epoch {epoch}")
import sys

def load_checkpoint(epoch, model, optimizer,scheduler,args):
    filename=f"checkpoint_epoch_{epoch}.pth.tar"
    dir=f"{args.save_dir}"
    if args.encoder_mode== True:
        dir=f"{args.save_dir}/encoder"
    checkpoint = torch.load(f"{dir}/{filename}")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # epoch = checkpoint['epoch']
    # args = checkpoint['args']
    scheduler.load_state_dict(checkpoint['scheduler'])
    print(f"Checkpoint loaded from epoch {epoch}")
    return epoch + 1, args,model,scheduler,optimizer
def train_one_epoch(model, train_loader, optimizer,scheduler, device, epoch,demo=False,centroid_layer=False):
    model.train()
    total_loss = torch.zeros(1).to(device)
    train_loader = tqdm(train_loader, file=sys.stdout, ncols=100, colour='red')
    # optimizer.zero_grad()
    optimizer.zero_grad()
    accumulation_steps = 1  # Number of steps to accumulate gradients

    for i, data in enumerate(train_loader):
        # graph_data= build_batch_graph(data,device,centroid_layer=centroid_layer)
        # data=None
        # pred,label,pred_c,label_c = model(graph_data)
        if i!=-1: #for testing manual
            
            # data = data.to(device)
            
            graph_data= build_batch_graph(data,device,centroid_layer=centroid_layer)
            data=None
            # print(graph_data.exps.shape)
            # excep?t:
                # print(data)
            # graph_data.cpu()
            # data.cpu()
            pred,label,pred_c,label_c = model(graph_data)
            
            label = np.array(label, dtype=np.float32)
            label = torch.from_numpy(label)
            label= label.to(device)
            # print(label.shape,pred.shape)

            # print(type(label))
            if pred_c is not None:
                label_c = np.array(label_c, dtype=np.float32)
                label_c= torch.from_numpy(label_c)
                label_c= label_c.to(device)
                loss = F.mse_loss(pred, label)+ F.mse_loss(pred_c, label_c)
            else:
                loss = F.mse_loss(pred, label)

            loss.backward()
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                optimizer.step()  # Perform optimizer step
                scheduler.step()  # Adjust learning rate
                optimizer.zero_grad()
            
            # optimizer.step()
            # scheduler.step()
            # optimizer.zero_grad()
            
            total_loss = (total_loss * i + loss.detach()) / (i + 1)
            if i%3==0:
                train_loader.desc = 'Train\t[epoch {}] lr: {}\tloss {}'.format(epoch, optimizer.param_groups[0]["lr"], round(total_loss.item(), 3))
            # if i==3 and demo==True:
        #     break
    torch.cuda.empty_cache()
    return pred


@torch.no_grad()
def val_one_epoch(model, val_loader, device, centroid,demo=False, encoder_mode =False, data_type='val',centroid_layer=False):
    model.eval()
    labels = torch.tensor([], device=device)
    preds = torch.tensor([], device=device)
    if data_type == 'val':
        val_loader = tqdm(val_loader, file=sys.stdout, ncols=100, colour='blue')
    elif data_type == 'test':
        val_loader = tqdm(val_loader, file=sys.stdout, ncols=100, colour='green')

    for i, data in enumerate(val_loader):
        # data = data.to(device)
        graph_data= build_batch_graph(data,device,centroid_layer)
        # data.cpu()
        # graph_data.cpu()
        if encoder_mode ==True:
            centroid=0
        output,label,_,_= model(graph_data)
        output= output[centroid:]
        label= label[centroid:]
        
        label = torch.from_numpy(label).to(device)
        labels = torch.cat([labels.cpu(), label.cpu()], dim=0)
        preds = torch.cat([preds.cpu(), output.detach().cpu()], dim=0)
        # if i==3 and demo==True:
        #     break
    print(labels.shape,preds.shape)
    return preds.cpu(), labels.cpu()




In [ ]:
class GATModel_3(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=1024, output_dim=1024, num_heads=3,n_classes=460,centroid_layer=False):
        super(GATModel_3, self).__init__()

        # MLP for flattening emb_cells_in_cluster
       
        # GATConv for graph processing
        self.centroid_layer=centroid_layer
        self.gat_conv_centroid = GATv2Conv(input_dim, hidden_dim, heads=num_heads, concat=False)
        self.gat_conv = GATv2Conv(input_dim, hidden_dim, heads=num_heads, concat=False)
        self.activate = F.elu
        self.fc = nn.Linear(hidden_dim, n_classes)
    def forward(self, data):
        emb_data, exps, exps_c= data
        edge_index = emb_data.edge_index 
        h_c=None
        
        x=emb_data.x
      

        # Apply GATConv across the entire batch graph
        
        
        h = self.gat_conv(x, edge_index.T)
        h = self.fc(h).squeeze(0)
        
        # print(h.shape,exps.shape)
        return h,exps,h_c,exps_c

In [ ]:
# In the training function, users build and train the model to make inferences on the test data.
# Your model must be stored in the resources/ directory.
from sklearn.metrics import mean_squared_error, mean_absolute_error

def train(
    data_directory_path: str
    ,model_directory_path:str
):
    # no train
    no_train= False
    if no_train == True:
        return
    NAMES = ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']
    sdata_dir=data_directory_path
    # sdata_dir=f'F:/DATA/crunch_large/zip_server'
    pre_load_path= '/tmp/pre_load'
    os.makedirs(pre_load_path,exist_ok=True)
    create_pre_load(pre_load_path=pre_load_path,sdata_dir= sdata_dir,name_list=NAMES)
    cluster_path= '/tmp/cluster'
    create_cluster('train',n_centroid=4,NAMES=NAMES,pre_load_path=pre_load_path,cluster_path=cluster_path)
    split_cluster(cluster_path=cluster_path,NAMES=NAMES)

    crop_path='/tmp/patches'
    r=24/2
    crop_patches(sdata_dir=sdata_dir,pre_load_path=pre_load_path,crop_path=crop_path,r=r,name_list=NAMES)
    pretrain_model_path='/resources/pytorch_model.bin'
    preprocessed_path= '/tmp/preprocessed'
    tensor_folder = f'{crop_path}/{int(r*2)}'
    save_dir=f'{preprocessed_path}/{int(r*2)}'
    preprocess_patches(NAMES=NAMES,tensor_folder=tensor_folder
                              ,save_dir=save_dir
                                 ,resources_path=model_directory_path
                                  ,evel=False)

    args_dict = {
    'batch_size': 73,
    'epochs': 10,  # 90
    'seed': 2023,  # temperature
    'n_workers': 0,  # projection_dim
    'n_classes': 460,  # attention heads num
    'start_epoch': 0,  # attention heads dim
    'heads_layers': 2,  # attention heads layer num
    'dropout': 0.0,  # dropout
    'encoder_mode': False,  # dataset
    'demo': False,  # model saved path
    'resume': False,  # resume training
    'local': True,  # patch_size
    'centroid_layer': 'True',  # patch_size(n)-epoch(e)
    'embed_dir': f'{save_dir}',
    'demo': False,
    'device': 'cuda:0',
    'local': True,
    'save_dir': model_directory_path
}
    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')

    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    args = SimpleNamespace(**args_dict)
    train_NAMES= NAMES
    if args.demo== True:
        NAMES=NAMES[:2]
        train_NAMES=NAMES[:2]
    dir=args.embed_dir
    # dir='D:/DATA/Gene_expression/Crunch/preprocessed'
    if args.local == True:
        pin= False
    else:
        pin= True
    traindata= NeuronData_3(emb_folder=dir,train=True, split =True
                          ,name_list= train_NAMES
                          ,encoder_mode=args.encoder_mode
                          )
    train_dataLoader =DataLoader(traindata, batch_size=args.batch_size, shuffle=False,pin_memory=pin)    
    # print(len(train_dataLoader))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # traindata[2127]
    model=GATModel_3(centroid_layer=args.centroid_layer)
    model= model.to('cuda')
    #------------------------
    

    # print(f'Using fold {args.fold}')
    # print(f'valid: {len(val_set)}')

    
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-5)
    scheduler = LR_Scheduler(optimizer=optimizer
                             ,num_epochs=args.epochs
                             ,base_lr=0.00018
                             ,iter_per_epoch = len(train_dataLoader)
                             ,warmup_epochs= 10
                            ,warmup_lr= 0.00015
                            ,final_lr= 0.00005
                            ,constant_predictor_lr=False
)
    val_set= [NeuronData_3(emb_folder=dir,train=False, split =True,name_list= [name],encoder_mode=args.encoder_mode) 
              for name in NAMES]
    val_loader =[DataLoader(set, batch_size=args.batch_size, shuffle=False,pin_memory=pin)for set in val_set]    
    output_dir = args.save_dir
    
    # val_set = DATA_BRAIN(train=False,r=int(args.patch_size/2), device=args.device)
    # val_loader = DataLoader(val_set, batch_size=1024, num_workers=args.n_workers, shuffle=False)
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)


    print(f"Start training for {args.epochs} epochs")

  
    min_val_mse = 200.0
    min_val_mae = 200.0
    start_epoch= args.start_epoch
    if start_epoch >0:
        start_epoch, args, model,scheduler, optimizer = load_checkpoint(start_epoch, model, optimizer,scheduler,args)
    # for step in range(start_epoch*len(train_dataLoader)):
    #     scheduler.step()
    print(start_epoch)
    print(f'start epoch: {start_epoch}, batch size: {args.batch_size}')
    for epoch in range(start_epoch, args.epochs):
        checkpoint_filename = f"checkpoint_best_epoch_{epoch}.pth.tar"
        train_logits = train_one_epoch(model=model, train_loader=train_dataLoader
                                       ,demo=args.demo, optimizer=optimizer,scheduler=scheduler, device=device, epoch=epoch + 1,centroid_layer=args.centroid_layer)
        if (epoch+1)%8 ==0: 
           
            mse_list = []
            mae_list = []
            for index in range(len(val_loader)): 
                val_preds, val_labels = val_one_epoch(model=model,demo=args.demo
                                                      , val_loader=val_loader[index]
                                                      , device=device, data_type='val'
                                                      , centroid= args.batch_size
                                                      ,encoder_mode=args.encoder_mode
                                                      ,centroid_layer=args.centroid_layer)
                mse=mean_squared_error(val_labels, val_preds)
                mae=mean_absolute_error(val_labels, val_preds)
                ###############
                tmp_list=[str(i) for i in range(460)]
                val_labels=val_labels.numpy()
                val_preds= val_preds.numpy() 
                
                mse_list.append(mse)
                mae_list.append(mae)
            
          
        if (epoch+1)%4==0:
            checkpoint_filename = f"checkpoint_epoch_{epoch}.pth.tar"
            save_checkpoint(epoch, model, optimizer,scheduler, args, filename=checkpoint_filename)

    

## TEST

In [ ]:
def load_checkpoint_test(epoch, model,args):
    try:
        filename=f"checkpoint_epoch_{epoch}.pth.tar"
        dir=f"{args.save_dir}"
        checkpoint = torch.load(f"{dir}/{filename}")
    except:
        filename=f"checkpoint_epoch_best_{epoch}.pth.tar"
        dir=f"{args.save_dir}"
        checkpoint = torch.load(f"{dir}/{filename}")
    if args.encoder_mode== True:
        dir=f"{args.save_dir}"
    model.load_state_dict(checkpoint['model_state_dict'])
    # epoch = checkpoint['epoch']
    # args = checkpoint['args']
    # scheduler.load_state_dict(checkpoint['scheduler'])
    print(f"Checkpoint loaded from epoch {epoch}")
def evel_slide(model, val_loader, device, centroid,demo=False, encoder_mode =False, data_type='val',centroid_layer=False):
    model.eval()
    preds = torch.tensor([], device=device)
    if data_type == 'val':
        val_loader = tqdm(val_loader, file=sys.stdout, ncols=100, colour='blue')
    elif data_type == 'test':
        val_loader = tqdm(val_loader, file=sys.stdout, ncols=100, colour='green')

    for i, data in enumerate(val_loader):
        # data = data.to(device)
        graph_data= build_batch_graph(data,device,centroid_layer)
        # data.cpu()
        # graph_data.cpu()
        if encoder_mode ==True:
            centroid=0
        output,_,_,_= model(graph_data)
        output= output[centroid:]
        
        
        preds = torch.cat([preds.cpu(), output.detach().cpu()], dim=0)
        # if i==3 and demo==True:
        #     break
    print(preds.shape)
    return preds.cpu()


In [ ]:
# In the inference function, the trained model is loaded and used to make inferences on a
# sample of data that matches the characteristics of the training test.
import gc
def infer(
    data_file_path: str,
    model_directory_path: str
):
    name=os.path.splitext(os.path.basename(data_file_path))[0]
    sdata_dir=os.path.dirname(data_file_path)
    print(data_file_path, name,sdata_dir)


    preprocessed_path= '/tmp/preprocessed'
    pre_load_path= '/tmp/pre_load'
    os.makedirs(pre_load_path,exist_ok=True)
    NAMES= ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']
    create_pre_load(pre_load_path=pre_load_path,name_list=NAMES,sdata_dir= sdata_dir)

    cluster_path= '/tmp/cluster'
    create_cluster('evel',n_centroid=600,NAMES=[name],pre_load_path=pre_load_path,cluster_path=cluster_path)
    crop_path='/tmp/patches'
    r=24/2
    crop_patches(sdata_dir=sdata_dir,pre_load_path=pre_load_path,crop_path=crop_path,r=r,evel=True,name_list=[name])
    pretrain_model_path='/resources/pytorch_model.bin'
    tensor_folder = f'{crop_path}/{int(r*2)}'
    save_dir=f'{preprocessed_path}/{int(r*2)}'
    preprocess_patches(NAMES=[name],tensor_folder=tensor_folder
                       ,save_dir=save_dir
                       ,resources_path=model_directory_path
                       ,evel=True)

    args_dict = {
    'batch_size': 73,
    'epochs': 10,  # 90
    'seed': 2023,  # temperature
    'n_workers': 0,  # projection_dim
    'n_classes': 460,  # attention heads num
    'start_epoch': 0,  # attention heads dim
    'heads_layers': 2,  # attention heads layer num
    'dropout': 0.0,  # dropout
    'encoder_mode': False,  # dataset
    'demo': False,  # model saved path
    'resume': False,  # resume training
    'local': True,  # patch_size
    'centroid_layer': 'True',  # patch_size(n)-epoch(e)
    'embed_dir': f'{save_dir}',
    'demo': False,
    'device': 'cuda:0',
    'local': True,
    'save_dir': model_directory_path
}
    args = SimpleNamespace(**args_dict)
    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')

    # NAMES= ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']
    
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    dir=args.embed_dir
 
    model=GATModel_3()
    model= model.to(device)
    val_set= NeuronData_3(emb_folder=dir,cluster_path=cluster_path,train=False, split =True,name_list= [name],encoder_mode=args.encoder_mode,evel=True) 
             
    val_loader =DataLoader(set, batch_size=args.batch_size, shuffle=False,pin_memory=True)
    start_epoch= args.start_epoch
    start_epoch, args, optimizer = load_checkpoint(start_epoch, model, optimizer,args)
    preds = val_one_epoch(model=model,demo=args.demo
                                                , val_loader=val_loader
                                                , device=device, data_type='val'
                                                , centroid= args.batch_size
                                                ,encoder_mode=args.encoder_mode)
    sdata = sd.read_zarr(data_file_path)
    # gene_names = sdata["anucleus"].var.index
    try:
      gene_names = sdata["anucleus"].var.index
    except:
      gene_names = sd.read_zarr('/data/DC5.zarr')["anucleus"].var.index

    sdata=None
    
    cell_ids = []
    for idx in range(len(val_set)):
        data = val_set[idx]  # Access the data at index `idx`
        cell_ids.extend(data.cell_ids.tolist())
    pred = np.round(pred, 2)

    prediction = pd.DataFrame(
        itertools.product(cell_ids, gene_names),
        columns=["cell_id", "gene"]
    )

    prediction["prediction"] = pred.ravel(order="C")
    del pred, sdata
    return prediction


In [ ]:
# This command is running a local test with your submission
# # making sure that your submission can be accepted by the system
crunch.test(
    no_determinism_check=True,
)

Now remember to download this notebook and then submit it at https://hub.crunchdao.com/competitions/broad-1/submit/